In [3]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from geodatasets import get_path 

In [4]:

import re

from geopy.geocoders import ArcGIS
#from geopy.extra.rate_limiter import RateLimiter

import plotly.express as px 


#import nltk

import spacy
from spacy import displacy


In [5]:
#nlp = spacy.load("en_core_web_lg")

In [6]:
text = "corrido corpus/ElCorridodeGregorioCortez_X.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_ner_model")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)

['Karnes', 'Gonzales', 'Belmont', 'Laredo', 'Encinal']


In [7]:
ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
print(ents)

ents = [(e.text)for e in doc.ents]
print(ents)

[('Karnes', 18, 24, 'GPE'), ('Gonzales', 1051, 1059, 'GPE'), ('Belmont', 1140, 1147, 'GPE'), ('Laredo', 1550, 1556, 'GPE'), ('Encinal', 1864, 1871, 'GPE')]
['Karnes', 'Gonzales', 'Belmont', 'Laredo', 'Encinal']


In [8]:
displacy.render(doc, style ='ent', jupyter=True, page=True)

Geoparsing: Finding places from the corrido lyrics

In [45]:
df = pd.DataFrame(ents, columns=["NER_Places"])
geolocator = ArcGIS(user_agent='CorridosMap')
geocode = lambda query: geolocator.geocode("%s, Texas" % query)
df['NER_Places'] = df['NER_Places'].replace('Belmont', 'Belmont Site, Gonzales County')
df['Location'] = df['NER_Places'].apply(geocode)
#df['geometry'] = df['NER_Places'].apply(geocode)
df.dtypes
df

,NER_Places,Location
0,Karnes,"(Karnes County, Texas, (28.90574, -97.8594))"
1,Gonzales,"(Gonzales, Texas, (29.5155311, -97.4631481))"
2,"Belmont Site, Gonzales County","(Belmont, Texas, (29.5239597, -97.6865508))"
3,Laredo,"(Laredo, Texas, (27.5071293, -99.5074957))"
4,Encinal,"(Encinal, Texas, (28.041054, -99.354971))"


In [10]:
df.NER_Places

0                           Karnes
1                         Gonzales
2    Belmont Site, Gonzales County
3                           Laredo
4                          Encinal
Name: NER_Places, dtype: object

In [ ]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326"
)

In [48]:
gdf = gpd.tools.geocode(df.Location)

gdf

,geometry,address
0,POINT (-97.90629 28.87919),"Karnes, Texas, United States"
1,POINT (-97.51086 29.44366),"Gonzales, Texas, United States"
2,POINT (-97.68751 29.52411),"Belmont, 78122, Texas, United States"
3,POINT (-99.50745 27.50563),"Laredo, Texas, United States"
4,POINT (-99.35521 28.04063),"Encinal, Texas, United States"


In [12]:
gdf = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf["lat"]=gdf['geometry'].y
gdf ["lon"] = gdf['geometry'].x

gdf

,geometry,address,lat,lon
0,POINT (-97.90629 28.87919),"Karnes, Texas, United States",28.879186,-97.906289
1,POINT (-97.51086 29.44366),"Gonzales, Texas, United States",29.443655,-97.510864
2,POINT (-97.68751 29.52411),"Belmont, 78122, Texas, United States",29.524107,-97.687512
3,POINT (-99.50745 27.50563),"Laredo, Texas, United States",27.505625,-99.507452
4,POINT (-99.35521 28.04063),"Encinal, Texas, United States",28.040633,-99.355210


In [22]:
px.set_mapbox_access_token(open("mapboxtoken").read())
fig = px.line_mapbox(gdf,
                        lat=gdf.geometry.y,
                        lon=gdf.geometry.x,
                        hover_name="address",
            
                        )

fig.update_layout(mapbox_style="open-street-map", mapbox_zoom=6, mapbox_center_lat = 29,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()